In [1]:
# Get Imports
import pandas as pd

In [2]:
import json

filepath = "./spotify_million_playlist_dataset/data"
filename = filepath + "/mpd.slice.0-999.json"

MAXPLAYLISTSIZE = 999999
MAXREADFILES = 10000
PLAYLISTPERFILE = 1000

# Input: filesToRead(int) number of files to read
# Add error handling later
def readFile(filesToRead):
    playlistNames = []
    playlistTracksIds = []
    playlistArtistIds = []
    playlistAlbumIds = []
    for slice_number in range(filesToRead):
        filename = filepath + "/mpd.slice." + str(slice_number*PLAYLISTPERFILE) + "-" + str((slice_number*PLAYLISTPERFILE)+PLAYLISTPERFILE-1) + ".json"
        print(filename)
        
        with open(filename, 'r') as file:
            playlist_data = json.load(file)

        listOfPlayLists = playlist_data['playlists']

        for playlist in listOfPlayLists:
            playlistNames.append(playlist['name'])
            trackIds = [track['track_uri'] for track in playlist['tracks']]
            playlistTracksIds.append(trackIds)
            artistIds = [artist['artist_uri'] for artist in playlist['tracks']]
            playlistArtistIds.append(artistIds)
            albumIds = [album['album_uri'] for album in playlist['tracks']]
            playlistAlbumIds.append(albumIds)
    return pd.DataFrame({'name': playlistNames, 'trackIds': playlistTracksIds, 'artistIds': playlistArtistIds})

In [3]:
# Get data 
playListDF = readFile(10)

./spotify_million_playlist_dataset/data/mpd.slice.0-999.json
./spotify_million_playlist_dataset/data/mpd.slice.1000-1999.json
./spotify_million_playlist_dataset/data/mpd.slice.2000-2999.json
./spotify_million_playlist_dataset/data/mpd.slice.3000-3999.json
./spotify_million_playlist_dataset/data/mpd.slice.4000-4999.json
./spotify_million_playlist_dataset/data/mpd.slice.5000-5999.json
./spotify_million_playlist_dataset/data/mpd.slice.6000-6999.json
./spotify_million_playlist_dataset/data/mpd.slice.7000-7999.json
./spotify_million_playlist_dataset/data/mpd.slice.8000-8999.json
./spotify_million_playlist_dataset/data/mpd.slice.9000-9999.json


In [10]:
# Get all the songs 
explodedTracks = playListDF.explode('trackIds')
uniqueTracks = explodedTracks['trackIds'].unique()
# Construct a binary matrix with rows=playlists and columns=songs
tracksMatrix = pd.DataFrame(0, index=range(len(playListDF)), columns=uniqueTracks)
for i, playlist in playListDF.iterrows():
    for tracks in playlist['trackIds']:
        tracksMatrix.loc[i, tracks] = 1



In [22]:
# Reduce the size of the matrix
from scipy.sparse import csr_matrix, coo_matrix
tracksMatrixSparse = csr_matrix(tracksMatrix.values)
tracksMatrixCOO = tracksMatrixSparse.tocoo()
tracksMatrixCOO = pd.DataFrame({'playlistId': tracksMatrixCOO.row, 'trackId':tracksMatrixCOO.col, 'inPlaylist':tracksMatrixCOO.data})

In [45]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(tracksMatrixCOO, reader)

trainset, testset = train_test_split(data)

model = SVD(biased=True, n_epochs=30, lr_all=0.01, reg_all=0.1, verbose=True)
model.fit(trainset)

predictions = model.test(testset)

accuracy.rmse(predictions)



Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
RMSE: 0.0107


0.010715237242704812

In [42]:
def getTopPredictions(model, playlist_id, numPredictions):
    # Get all item IDs in the dataset
    item_ids = set([iid for (_, iid, _) in trainset.all_items()])

    # Predict ratings for all items for the given user
    predictions = [(iid, model.predict(playlist_id, iid).est) for iid in item_ids]

    # Sort the predictions by predicted rating in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Extract the top-N item IDs from the sorted predictions
    top_n = [iid for (iid, _) in predictions[:numPredictions]]
    return top_n

In [43]:
def calculate_precision(recommended_items, ground_truth):
    # Calculate the number of overlapping items between recommended items and ground truth
    overlapping_items = set(recommended_items) & set(ground_truth)
    
    # Calculate precision: Number of overlapping items / Total number of recommended items
    if len(recommended_items) > 0:
        precision = len(overlapping_items) / len(recommended_items)
    else:
        precision = 0.0  # Handle the case when there are no recommended items

    return precision

In [44]:
# Lets test using R-precision as used in the Spotify Million Dollar Playset Challenge
# Get the list of unique user IDs in the test set
playlistIds = set([uid for (uid, _, _) in testset])

# Initialize variables to store precision values
total_precision = 0
total_users = 0

# Calculate R-Precision for each user in the test set
for playlistId in playlistIds:
    # Get the list of items the user has interacted with (ground truth)
    ground_truth = [iid for (uid, iid, _) in testset if uid == playlistId]

    # Generate recommendations for the user
    recommendations = getTopPredictions(model, playlistId, 10)

    # Calculate precision by comparing the recommended items with the ground truth
    precision = calculate_precision(recommendations, ground_truth)

    # Update total precision and total users
    total_precision += precision
    total_users += 1

# Calculate average precision (R-Precision)
r_precision = total_precision / total_users

TypeError: cannot unpack non-iterable int object

AttributeError: 'list' object has no attribute 'shape'